In [1]:
from langchain.document_loaders import WikipediaLoader
from langchain.docstore.document import Document
from modules.knowledge_graph import diffbot_nlp
from modules.neo4j import graph_chain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI


In [2]:
graph_chain.graph.refresh_schema()

In [3]:
graph_chain.graph.get_schema

"\n        Node properties are the following:\n        [{'labels': 'Node', 'properties': [{'property': 'type', 'type': 'STRING'}, {'property': 'name', 'type': 'STRING'}]}]\n        Relationship properties are the following:\n        []\n        The relationships are the following:\n        ['(:Node)-[:EDUCATE_AT]->(:Node)', '(:Node)-[:LOCATE_AT]->(:Node)']\n        "

In [4]:
graph_chain.run("where is abc")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Node)-[:LOCATE_AT]-(o:Node) WHERE apoc.text.phonetic(p.name) = apoc.text.phonetic("abc") RETURN o;
Full Context:
[{'o': {'name': 'Hanoi', 'type': 'location'}}, {'o': {'name': 'Hanoi', 'type': 'location'}}]

> Finished chain.


'I\'m sorry, but I don\'t have enough information to answer your question. Can you please provide more details or clarify what you mean by "abc"?'

In [37]:
llm_chain.run({ 
  "question": "John is a tall guy who ages 20 years old, everybody likes him"
})

'John,age,20\nJohn,is_tall\neveryone,LOVES,John'

In [1]:
from dotenv import load_dotenv
load_dotenv()

import faiss

from datetime import datetime
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS


embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = OpenAIEmbeddings().embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})

retriever = vectorstore.as_retriever(search_kwargs=dict(k=1))
memory = VectorStoreRetrieverMemory(retriever=retriever)

# When added to an agent, the memory object can save pertinent information from conversations or used tools
memory.save_context({"input": "My favorite food is pizza"}, {"output": "that's good to know"})
memory.save_context({"input": "My favorite sport is soccer"}, {"output": "..."})
memory.save_context({"input": "I don't the Celtics"}, {"output": "ok"}) #


input: My favorite sport is soccer
output: ...


In [2]:
print(memory.load_memory_variables({"prompt": "My favorite food"})["history"])


input: My favorite food is pizza
output: that's good to know
